ファイル構造
データ置き場
在庫MB
所在管理M

表記揺れを課題として書く！

In [1]:
#メモリのリセット
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
#注意）ファイル名表示されない
%ls

 Volume in drive C has no label.
 Volume Serial Number is 6EFA-366C

 Directory of C:\Users\1082794-Z100\Desktop\???)AI???????

2023/12/26  20:43    <DIR>          .
2023/12/26  20:43    <DIR>          ..
2023/12/22  09:05    <DIR>          .ipynb_checkpoints
2023/12/21  17:02             4,311 ??.ipynb
2023/12/21  16:26         3,845,634 ?�????.ipynb
2023/12/22  11:04           246,660 ??�???.ipynb
2023/12/26  19:44    <DIR>          ?�????
2023/12/26  20:43            70,800 ???.ipynb
2023/12/21  16:36           751,260 ???&???.ipynb
2023/12/22  10:33    <DIR>          ?????
               5 File(s)      4,918,665 bytes
               5 Dir(s)  67,898,781,696 bytes free


In [52]:
#ライブラリのimport
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re 
from dateutil.relativedelta import relativedelta

In [172]:
#元データのパス
folder_path_zaikoMB = 'データ置き場/在庫MB'
folder_path_LTMB = 'データ置き場/所在管理MB'
folder_path_tehaisu = 'データ置き場/手配必要数'
folder_path_tehaiunyo = 'データ置き場/手配運用情報'
file_path_pitch = 'データ置き場/不等ピッチ係数/不等ピッチ係数.csv'
folder_path_kumitate = 'データ置き場/組立実績MB'
#中間成果物のパス
folder_path_interproduct = '中間成果物'
file_path_zaikodata = '中間成果物/在庫MBデータ_統合済.csv'
file_path_LTdata = '中間成果物/所在管理MBデータ_統合済.csv'
file_path_LTdata_extract = '中間成果物/所在管理MBデータ_統合済&特定日時抽出済.csv'
file_path_tehaisu_with_tehaiunyo = '中間成果物/手配数データ_手配運用情報統合済'
file_path_LTdata_extract_with_tehaisu = '中間成果物/所在管理MBデータ_統合済&特定日時抽出済&手配数と手配運用情報統合済.csv'
file_path_weekly_data = '中間成果物/週単位のデータ.csv'
file_path_kumitate = '中間成果物/組立実績MB_統合済.csv'

在庫推移MotionBoardよりデータ出力

■目的

■在庫推移MotionBoard
http://10.88.11.58:18081/MBSF/main?id=b9154fd0-6fe6-d153-7d17-053da4c333ea&mbid=fidtnjir4nq3ja63cqcaaafmoqcx4&boardpath=%2FMSR%2F%E8%87%AA%E5%8B%95%E3%83%A9%E3%83%83%E3%82%AF%2FT403%E7%89%A9%E6%B5%81%E6%83%85%E5%A0%B1_%E5%9C%A8%E5%BA%AB%E6%8E%A8%E7%A7%BB

In [6]:
# フォルダー内の全てのCSVファイルを見つける
csv_files_zaikoMB = [f for f in os.listdir(folder_path_zaikoMB) if f.endswith('.csv')]
#
print(f"{len(csv_files_zaikoMB)}つのファイルが見つかりました！")

# 統合結果を保存するための空のDataFrameを作成
merged_df_zaiko = pd.DataFrame()
# CSVファイルをDataFrameに読み込んでリストに保存
for file in csv_files_zaikoMB:
    file_path = os.path.join(folder_path_zaikoMB, file)
    df_zaiko = pd.read_csv(file_path, encoding='Shift_JIS')
    merged_df_zaiko = pd.concat([merged_df_zaiko, df_zaiko], ignore_index=True)

# 統合したデータを新しいCSVファイルに保存
with open(file_path_zaikodata, mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_df_zaiko.to_csv(f)

#作成したDataFrameの内容を確認
display(merged_df_zaiko)

2つのファイルが見つかりました！


,計測日時,品番,拠点所番地,在庫数（箱）,入庫数（箱）,出庫数（箱）
0,2023/12/10 00:00:00,019120LC030,30113,11.0,0.0,0.0
1,2023/12/10 00:00:00,019120LC050,20115,5.0,0.0,0.0
2,2023/12/10 00:00:00,019120LC060,30116,10.0,0.0,0.0
3,2023/12/10 00:00:00,019120LC070,20117,11.0,0.0,0.0
4,2023/12/10 00:00:00,019128GA010,50460,26.0,0.0,0.0
...,...,...,...,...,...,...
95627,2023/12/15 18:00:00,G9201ECE010,50446,55.0,0.0,2.0
95628,2023/12/15 18:00:00,G9351ECB010,20409,25.0,0.0,4.0
95629,2023/12/15 18:00:00,G9351ECE010,20406,1.0,0.0,0.0
95630,2023/12/15 18:00:00,Y021720076*****,30121,NaN,NaN,NaN


所在管理MOtionBoardよりデータ出力

➀「サブフィルターを表示」押下
➁1Yを設定
➂SHIftjisでダウンロード

■所在管理MotionBoard
http://10.88.11.58:18081/MBSF/main?id=8045ec22-9e62-92c7-5e1e-077d35e5dc33&mbid=fidepnyfuff3ra63kitaaae3ywjmi&boardpath=%2FMSR%2F%E8%87%AA%E5%8B%95%E3%83%A9%E3%83%83%E3%82%AF%2FT403%E7%89%A9%E6%B5%81%E6%83%85%E5%A0%B1_v2

In [7]:
# フォルダー内の全てのCSVファイルを見つける
csv_files_LTMB = [f for f in os.listdir(folder_path_LTMB) if f.endswith('.csv')]
#
print(f"{len(csv_files_LTMB)}つのファイルが見つかりました！")

# 統合結果を保存するための空のDataFrameを作成
merged_df_LT = pd.DataFrame()
# CSVファイルをDataFrameに読み込んでリストに保存
for file in csv_files_LTMB:
    file_path = os.path.join(folder_path_LTMB, file)
    df_LT = pd.read_csv(file_path, encoding='Shift_JIS',dtype=str)
    merged_df_LT = pd.concat([merged_df_LT, df_LT], ignore_index=True)

# 不要な特定の列を削除
merged_df_LT = merged_df_LT.drop("伝票番号", axis=1)
merged_df_LT = merged_df_LT.drop("拠点所番地", axis=1)
merged_df_LT = merged_df_LT.drop("組立投入日時", axis=1)
merged_df_LT = merged_df_LT.drop("発注取消日時", axis=1)
merged_df_LT = merged_df_LT.drop("発注〜組立LT", axis=1)
merged_df_LT = merged_df_LT.drop("長期滞留フラグ", axis=1)

# 特定の文字列を含む行を削除する
target_string = "< NULL >"
merged_df_LT = merged_df_LT[~merged_df_LT.apply(lambda x: x.str.contains(target_string)).any(axis=1)]

# 統合したデータを新しいCSVファイルに保存
with open(file_path_LTdata, mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_df_LT.to_csv(f)

#作成したDataFrameの内容を確認
display(merged_df_LT)

1つのファイルが見つかりました！


,かんばんシリアル,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,納入便,発注日時,...,検収日時,順立装置入庫日時,順立装置出庫日時,回収日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜回収LT,更新日時
152,1Z23M00045678,35441ECB010,ｽﾄﾚｰﾅｰｵｲﾙ,48,（株）ニフコ,名古屋事業所,回収済,2023/12/20,01,2023/12/18 09:59:03,...,2023/12/20 15:28:10,2023/12/20 23:53:42,2023/12/21 09:34:31,2023/12/21 12:14:11,0.09,2.23,2.58,2.98,3.09,2023/12/21 12:14:11
153,1Z23M00045315,35882ECB010,ﾁｭｰﾌﾞﾃﾞﾌｷﾞﾔﾘｭｰﾌﾞｱﾌﾟﾗｲ,50,（株）メタルテック,NaN,回収済,2023/12/20,01,2023/12/16 06:47:00,...,2023/12/20 07:13:21,2023/12/20 09:27:18,2023/12/21 06:39:56,2023/12/21 12:14:11,2.09,4.02,4.11,5,5.23,2023/12/21 12:14:11
154,1Z23M00043118,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,NaN,回収済,2023/12/20,01,2023/12/18 10:25:45,...,2023/12/20 10:14:48,2023/12/20 14:59:47,2023/12/21 10:20:52,2023/12/21 12:14:11,0.15,1.99,2.19,3,3.08,2023/12/21 12:14:11
155,1Z23M00042507,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,NaN,回収済,2023/12/20,01,2023/12/18 10:25:45,...,2023/12/20 10:14:48,2023/12/20 15:00:48,2023/12/21 09:52:11,2023/12/21 12:14:11,0.15,1.99,2.19,2.98,3.08,2023/12/21 12:14:11
156,1Z23M00041030,35847ECB020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,70,（株）メタルテック,NaN,回収済,2023/12/19,01,2023/12/15 06:42:17,...,2023/12/19 07:16:04,2023/12/19 09:31:40,2023/12/21 04:59:21,2023/12/21 12:14:11,0.09,4.02,4.12,5.93,6.23,2023/12/21 12:14:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276028,1Z23G00047254,G1250ECE010,"ﾊﾟｲﾌﾟASSY,ﾓｰﾀｰｸｰﾘﾝｸﾞ",60,（株）バルカー,豊田営業所,回収済,2023/07/21,01,2023/07/18 03:42:22,...,2023/07/21 09:17:16,2023/07/21 15:19:33,2023/07/24 11:08:56,2023/07/24 22:54:33,0.38,3.23,3.48,6.31,6.8,2023/08/30 11:13:00
276029,1Z23G00047253,G1250ECB010,"ﾊﾟｲﾌﾟASSY,ﾓｰﾀｰｸﾘｰﾆﾝｸﾞNO.1",60,（株）バルカー,豊田営業所,回収済,2023/07/21,01,2023/07/18 03:42:22,...,2023/07/21 09:17:16,2023/07/21 15:19:49,2023/07/22 04:47:29,2023/07/24 15:20:42,0.38,3.23,3.48,4.05,6.48,2023/08/30 11:13:00
276395,1Z23G00046521,G1250ECE010,"ﾊﾟｲﾌﾟASSY,ﾓｰﾀｰｸｰﾘﾝｸﾞ",60,（株）バルカー,豊田営業所,回収済,2023/07/21,01,2023/07/18 03:42:22,...,2023/07/21 09:17:16,2023/07/21 15:19:25,2023/07/22 04:30:37,2023/07/24 15:20:43,0.38,3.23,3.48,4.03,6.48,2023/08/30 11:13:00
276600,1Z23G00046204,G1250ECE010,"ﾊﾟｲﾌﾟASSY,ﾓｰﾀｰｸｰﾘﾝｸﾞ",60,（株）バルカー,豊田営業所,回収済,2023/07/21,01,2023/07/18 03:42:22,...,2023/07/21 09:17:16,2023/07/21 15:19:37,2023/07/27 22:52:24,2023/07/28 04:17:23,0.38,3.23,3.48,9.8,10.02,2023/08/30 11:13:00


手配数

ファイル閉じないと書き込みできない

In [65]:
# ユーザーからの入力を受け付けて特定の日付範囲のDataFrameを取り出す
start_date_input = input("開始日を入力してください（YYYY-MM-DD）: ")
end_date_input = input("終了日を入力してください（YYYY-MM-DD）: ")

##########所在管理MBデータ

# datetime型に変換する
merged_df_LT['回収日時'] = pd.to_datetime(merged_df_LT['回収日時'], errors='coerce')
merged_df_LT['納入日'] = pd.to_datetime(merged_df_LT['納入日'], errors='coerce') 

# 入力された日付をdatetime型に変換
start_date = pd.to_datetime(start_date_input).date()
end_date = pd.to_datetime(end_date_input).date()

# 特定の日付範囲のDataFrameを取り出す
merged_df_LT2 = merged_df_LT[(merged_df_LT["回収日時"].dt.date >= start_date) & (merged_df_LT["回収日時"].dt.date <= end_date)]

#インデックスを振り直す
merged_df_LT2.reset_index(drop=True, inplace=True)

# 統合したデータを新しいCSVファイルに保存
with open(file_path_LTdata_extract, mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_df_LT2.to_csv(f)

display(merged_df_LT2)

開始日を入力してください（YYYY-MM-DD）:  2023-11-20
終了日を入力してください（YYYY-MM-DD）:  2023-12-10


,かんばんシリアル,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,納入便,発注日時,...,検収日時,順立装置入庫日時,順立装置出庫日時,回収日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜回収LT,更新日時
0,1Z23M00016940,9036340A010,ﾍﾞｱﾘﾝｸﾞ ﾗｼﾞｱﾙﾎﾞｰﾙ,80,日本精工（株）,三河分室,回収済,2023-12-08,02,2023/12/07 06:00:11,...,2023/12/08 11:18:39,2023/12/08 14:38:03,2023/12/09 03:55:59,2023-12-09 06:45:41,0.06,1.22,1.36,1.91,2.03,2023/12/09 06:45:41
1,1Z23M00015194,9036340A010,ﾍﾞｱﾘﾝｸﾞ ﾗｼﾞｱﾙﾎﾞｰﾙ,80,日本精工（株）,三河分室,回収済,2023-12-07,07,2023/12/06 18:06:56,...,2023/12/08 03:38:46,2023/12/08 05:33:54,2023/12/08 23:52:00,2023-12-09 06:45:41,0.27,1.4,1.48,2.24,2.53,2023/12/09 06:45:41
2,1Z23M00012463,82824ECB010,ﾜｲﾔﾘﾝｸﾞﾊｰﾈｽｺﾈｸﾀ,32,矢崎総業（株）,NaN,回収済,2023-12-08,01,2023/12/06 10:42:00,...,2023/12/08 10:18:18,2023/12/08 14:17:57,2023/12/09 03:36:05,2023-12-09 06:45:41,0.14,1.98,2.15,2.7,2.84,2023/12/09 06:45:41
3,1Z23M00011795,35847ECB010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,80,（株）メタルテック,NaN,回収済,2023-12-08,01,2023/12/06 06:45:37,...,2023/12/08 07:28:01,2023/12/08 09:53:01,2023/12/09 00:37:03,2023-12-09 06:45:41,0.09,2.03,2.13,2.74,3,2023/12/09 06:45:41
4,1Z23M00010125,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,NaN,回収済,2023-12-07,01,2023/12/05 10:41:25,...,2023/12/07 10:17:29,2023/12/07 14:24:18,2023/12/09 04:14:26,2023-12-09 06:45:41,0.14,1.98,2.15,3.73,3.84,2023/12/09 06:45:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30614,1Z23L00047027,35580ECB011,"ｱｷｭｴｰﾀｰASSY,ｼﾌﾄ ｺﾝﾄﾛｰﾙ",6,アイシン精機（株）,半田工場,回収済,2023-11-16,03,2023/11/15 16:35:21,...,2023/11/16 20:47:20,2023/11/17 11:19:15,2023/11/18 04:06:08,2023-11-20 09:03:51,0.21,1.17,1.78,2.48,4.69,2023/11/20 09:03:51
30615,1Z23L00047021,35580ECB011,"ｱｷｭｴｰﾀｰASSY,ｼﾌﾄ ｺﾝﾄﾛｰﾙ",6,アイシン精機（株）,半田工場,回収済,2023-11-16,03,2023/11/15 16:35:21,...,2023/11/16 20:47:20,2023/11/17 11:17:51,2023/11/18 04:50:37,2023-11-20 09:03:51,0.21,1.17,1.78,2.51,4.69,2023/11/20 09:03:51
30616,1Z23L00045676,35580ECB011,"ｱｷｭｴｰﾀｰASSY,ｼﾌﾄ ｺﾝﾄﾛｰﾙ",6,アイシン精機（株）,半田工場,回収済,2023-11-16,02,2023/11/15 10:51:30,...,2023/11/16 14:48:43,2023/11/17 06:07:09,2023/11/18 04:27:31,2023-11-20 09:03:51,0.22,1.16,1.8,2.73,4.93,2023/11/20 09:03:51
30617,1Z23L00045674,35580ECB011,"ｱｷｭｴｰﾀｰASSY,ｼﾌﾄ ｺﾝﾄﾛｰﾙ",6,アイシン精機（株）,半田工場,回収済,2023-11-16,02,2023/11/15 10:51:30,...,2023/11/16 14:48:43,2023/11/17 06:06:32,2023/11/18 00:35:14,2023-11-20 09:03:51,0.22,1.16,1.8,2.57,4.93,2023/11/20 09:03:51


In [115]:
# フォルダー内の全てのCSVファイルを見つける
csv_files_tehaisu = [f for f in os.listdir(folder_path_tehaisu) if f.endswith('.csv')]
year_tehaisu = [file[2:4] for file in os.listdir(folder_path_tehaisu) if file.endswith(".csv")]
month_tehaisu = [file[4:6].replace("0", "") for file in os.listdir(folder_path_tehaisu) if file.endswith(".csv")]
#
print(f"{len(csv_files_tehaisu)}つのファイルが見つかりました！")

for i in range(len(year_tehaisu)):
    print(f"{year_tehaisu[i]}年の{month_tehaisu[i]}月のデータが見つかりました！") 

csv_files_tehaiunyo = [f for f in os.listdir(folder_path_tehaiunyo) if f.endswith('.csv')]
#
print(f"{len(csv_files_tehaiunyo)}つのファイルが見つかりました！")

pattern = r"手配運用情報(\d+)"
results = []
for file in csv_files_tehaiunyo:
    match = re.search(pattern, file)
    if match:
        results.append(match.group(1))
year_tehaiunyo = [file[0:2] for file in results]
month_tehaiunyo = [file[2:4].replace("0", "") for file in results]

for i in range(len(year_tehaiunyo)):
    print(f"{year_tehaiunyo[i]}年の{month_tehaiunyo[i]}月のデータが見つかりました！") 

year_diff = end_date.year - start_date.year
month_diff = end_date.month - start_date.month
total_months =  year_diff * 12 + month_diff + 1
print(total_months)

input_years = []
input_months = []

start_date_temp = start_date
for i in range(total_months):
    input_year = str(start_date_temp.year)[2:]
    input_month = start_date_temp.month
    start_date_temp = start_date_temp + relativedelta(months=i+1)
    print(input_year,input_month)
    
    input_years.append(input_year)
    input_months.append(input_month)
    
    flag = 0
    for j in range(len(csv_files_tehaisu)):
        if (int(input_year) == int(year_tehaisu[j])) and (int(input_month) == int(month_tehaisu[j])):
            flag = 1
    if flag == 1:
        print("手配数あります")
    elif flag == 0:
        print("手配数ありません")
        
    flag = 0
    for j in range(len(csv_files_tehaiunyo)):
        if (int(input_year) == int(year_tehaiunyo[j])) and (int(input_month) == int(month_tehaiunyo[j])):
            flag = 1
    if flag == 1:
        print("手配運用情報あります")
    elif flag == 0:
        print("手配運用情報ありません")

1つのファイルが見つかりました！
23年の12月のデータが見つかりました！
3つのファイルが見つかりました！
23年の8月のデータが見つかりました！
23年の11月のデータが見つかりました！
23年の12月のデータが見つかりました！
2
23 11
手配数ありません
手配運用情報あります
23 12
手配数あります
手配運用情報あります


In [129]:
year_match = []
month_match = []

index_tehaisu = 0
for file in csv_files_tehaisu:
    file_path = os.path.join(folder_path_tehaisu, file)
    tehaisudata = pd.read_csv(file_path, encoding='Shift_JIS',dtype=str)#手配数データを読み込んでDataframeを作成
    index_tehaiunyo = 0
    for file in csv_files_tehaiunyo:
        print(month_tehaisu[index_tehaisu],month_tehaiunyo[index_tehaiunyo])
        file_path = os.path.join(folder_path_tehaiunyo, file)
        tehaiunyodata = pd.read_csv(file_path, skiprows = 9, encoding='cp932',dtype=str)#手配数データを読み込んでDataframeを作成
        tehaiunyodata.columns = [col.replace("=","").replace('"','') for col in tehaiunyodata.columns]
        tehaiunyodata_cleaned = tehaiunyodata.applymap(lambda x: x.replace('=','').replace('"','') if isinstance(x, str) else x)
        if (year_tehaisu[index_tehaisu] == year_tehaiunyo[index_tehaiunyo]) and (month_tehaisu[index_tehaisu] == month_tehaiunyo[index_tehaiunyo]):
            long_tehaisu = tehaisudata.iloc[:,1]#手配数データの行長さを取得
            long_tehaiunyo = tehaiunyodata_cleaned.iloc[:,1]#手配運用データの行長さを取得
            for i in range(len(long_tehaisu)):
                hinban_tehaisu = merged_df_LT2.loc[i,'品番'].replace("-", "").replace(" ", "")#品番名から" "を削除 
                for j in range(len(long_tehaiunyo)):
                    hinban_tehaiunyo = tehaiunyodata_cleaned.loc[j,'品番'].replace('-', '').replace(' ', '')#品番名から" "を削除
                    if hinban_tehaisu == hinban_tehaiunyo:
                        tehaisudata.loc[i,"納入回数（間隔）"] = tehaiunyodata_cleaned.loc[j,"納入ｻｲｸﾙ.間隔"]
                        tehaisudata.loc[i,"納入回数（回数）"] = tehaiunyodata_cleaned.loc[j,"納入ｻｲｸﾙ.回数"]
                        tehaisudata.loc[i,"納入回数（遅れ）"] = tehaiunyodata_cleaned.loc[j,"納入ｻｲｸﾙ.情報"]
                        tehaisudata.loc[i,"箱種類"] = tehaiunyodata_cleaned.loc[j,"登録箱種"]
                        tehaisudata.loc[i,"箱重量"] = tehaiunyodata_cleaned.loc[j,"総重量(Kg)"]
        index_tehaiunyo = index_tehaiunyo + 1
    
    # 統合したデータを新しいCSVファイルに保存
    file_new = file_path_tehaisu_with_tehaiunyo + year_tehaisu[index_tehaisu] + month_tehaisu[index_tehaisu] + ".csv"
    with open(file_new, mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
        tehaisudata.to_csv(f)
    
    year_match.append(year_tehaisu[index_tehaisu])
    month_match.append(month_tehaisu[index_tehaisu])
        
    index_tehaisu = index_tehaisu + 1
    
    display(tehaisudata)

12 8
12 11
12 12


,品番,加工図符号,設変符号,品名,ステータス,発注区分,整備室,整備室名,手配区分,工程内外製,...,25(火).1,26(水).1,27(木).1,28(金).1,29(土).1,30(日).1,納入回数（間隔）,納入回数（回数）,納入回数（遅れ）,箱重量
0,35300-ECB010,NaN,NaN,"PUMP ASSY, OIL W/MOTOR",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,957,895,957,895,0,0,1,08,9.00,9.301
1,35580-ECB011,NaN,A,"ACTUATOR ASSY, SHIFT CONTROL",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,429,391,435,397,0,0,1,08,9.00,9.301
2,1040 052 001Z,NaN,D,"PLUG, W/HEAD STRAIGHT SCREW",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,528,504,522,498,0,0,1,01,1.79,3.180
3,1040 183 011P,NaN,C,"BOLT, FLANGE",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,"1,287","1,173","1,305","1,191",0,0,1,01,1.90,2.900
4,3040 052 001B,NaN,C,"PLUG, W/HEAD STRAIGHT SCREW",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,"4,356","4,084","4,350","4,078",0,0,1,01,1.79,4.710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,G1259-ECB010,NaN,NaN,"RUBBER, MOTOR COOLING PIPE",使用中,かんばん,1Y,第１工場,直送,Ｐ,...,957,895,957,895,0,0,1,04,4.00,14.510
377,G1126-ECB010,NaN,A,"MAGNET, MOTOR ROTOR",使用中,かんばん,1Y,第１工場,直送,Ｐ,...,"12,240","11,280","12,528","11,568",0,0,1,04,4.00,14.510
378,G1126-ECE010,NaN,A,"MAGNET, MOTOR ROTOR",使用中,かんばん,1Y,第１工場,直送,Ｐ,...,"33,696","31,680","33,408","31,392",0,0,1,04,4.00,16.990
379,36296-ECB010,NaN,NaN,CAP,使用中,かんばん,1Y,第１工場,直送,Ｐ,...,"1,584","1,512","1,566","1,494",0,0,1,04,4.00,16.990


In [153]:
#中間成果物をダウンロード
merged_df_LT2 = pd.read_csv(file_path_LTdata_extract,encoding='shift_jis')

merged_df_LT2['回収日時'] = pd.to_datetime(merged_df_LT2['回収日時'], errors='coerce')
merged_df_LT2['回収月'] = merged_df_LT2['回収日時'].dt.month
merged_df_LT2['回収年'] = merged_df_LT2['回収日時'].dt.year
    
#index_file = 0
long = merged_df_LT2.iloc[:,1]
for m in range(total_months):
    input_year = input_years[m]
    input_month = input_months[m]
    file_path = file_path_tehaisu_with_tehaiunyo + str(input_year) + str(input_month) + ".csv"
    tehaisudata = pd.read_csv(file_path, encoding='Shift_JIS',dtype=str)#手配数データを読み込んでDataframeを作成
    for i in range(len(long)):
        if (int(str(merged_df_LT2.loc[i,'回収年'])[2:]) == int(input_years[m])) and (int(merged_df_LT2.loc[i,'回収月']) == int(input_months[m])):
            hinban = merged_df_LT2.loc[i,'品番']#品番名を取りだす 
            hinban2 = hinban.replace("-", "")#品番名から"-"を削除
            hinban3 = hinban2.replace(" ", "")#品番名から" "を削除 
            long_tehaisudata = tehaisudata.iloc[:,1]#手配数データの行長さを取得
            for j in range(len(long_tehaisudata)):
                tehaisuhinban = tehaisudata.loc[j,'品番'].replace('-', '')#品番名から"-"を削除
                tehaisuhinban2 = tehaisuhinban.replace(' ', '')#品番名から" "を削除
                if hinban3 == tehaisuhinban2:
                    merged_df_LT2.loc[i,"納入回数（間隔）"] = tehaisudata.loc[j,"納入回数（間隔）"]
                    merged_df_LT2.loc[i,"納入回数（回数）"] = tehaisudata.loc[j,"納入回数（回数）"]
                    merged_df_LT2.loc[i,"納入回数（遅れ）"] = tehaisudata.loc[j,"納入回数（遅れ）"]
                    merged_df_LT2.loc[i,"箱種類"] = tehaisudata.loc[j,"箱種類"]
                    merged_df_LT2.loc[i,"箱重量"] = tehaisudata.loc[j,"箱重量"]
                    merged_df_LT2.loc[i,"基準在庫日数"] = tehaisudata.loc[j,"基準在庫日数"]
                    merged_df_LT2.loc[i,"基準在庫枚数"] = tehaisudata.loc[j,"基準在庫枚数"]
                    day = merged_df_LT2.loc[i,'回収日時'].day
                    index = tehaisudata.columns.get_loc("翌々々月稼働日数") + day
                    merged_df_LT2.loc[i,'日量数'] = int(str(tehaisudata.iloc[j,index]).replace(',', ''))
                    merged_df_LT2.loc[i,'日量数（箱数）'] = int(merged_df_LT2.loc[i,'日量数']) /int(merged_df_LT2.loc[i, '収容数'])
                    merged_df_LT2.loc[i, '便Ave'] = merged_df_LT2.loc[i,'日量数（箱数）']/int(merged_df_LT2.loc[i, '納入回数（回数）'])
                    if int(str(tehaisudata.loc[j,'当月必要数']).replace(',', '')) != 0:
                        avenichiryo = int(str(tehaisudata.loc[j,'当月必要数']).replace(',', ''))/int(tehaisudata.loc[j,'当月稼働日数'])
                        merged_df_LT2.loc[i,'月平均日量数'] = avenichiryo
                        merged_df_LT2.loc[i,'月平均日量数（箱数）'] = avenichiryo//int(merged_df_LT2.loc[i, '収容数'])
                        merged_df_LT2.loc[i,'基準在庫枚数（日数）'] = (int(tehaisudata.loc[j,'基準在庫枚数'])*int(merged_df_LT2.loc[i, '収容数']))/avenichiryo
                        merged_df_LT2.loc[i,'基準在庫日数（基準在庫枚数込み）']= float(merged_df_LT2.loc[i,"基準在庫日数"]) + float(merged_df_LT2.loc[i,'基準在庫枚数（日数）'])
                    else:
                        merged_df_LT2.loc[i,'月平均日量数'] = 0
                        merged_df_LT2.loc[i,'月平均日量数（箱数）'] = 0
                        merged_df_LT2.loc[i,'基準在庫枚数（日数）'] = 0
                        merged_df_LT2.loc[i,'基準在庫日数（基準在庫枚数込み）']= 0
                        
    #index_file = index_file + 1

# 統合したデータを新しいCSVファイルに保存
with open(file_path_LTdata_extract_with_tehaisu, mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_df_LT2.to_csv(f)
    
display(merged_df_LT2)

,Unnamed: 0,かんばんシリアル,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,納入便,...,箱重量,基準在庫日数,基準在庫枚数,日量数,日量数（箱数）,便Ave,月平均日量数,月平均日量数（箱数）,基準在庫枚数（日数）,基準在庫日数（基準在庫枚数込み）
0,0,1Z23M00016940,9036340A010,ﾍﾞｱﾘﾝｸﾞ ﾗｼﾞｱﾙﾎﾞｰﾙ,80,日本精工（株）,三河分室,回収済,2023-12-08 00:00:00,2,...,2.288,0.76,3,0.0,0.000000,0.000000,1803.40,22.0,0.133082,0.893082
1,1,1Z23M00015194,9036340A010,ﾍﾞｱﾘﾝｸﾞ ﾗｼﾞｱﾙﾎﾞｰﾙ,80,日本精工（株）,三河分室,回収済,2023-12-07 00:00:00,7,...,2.288,0.76,3,0.0,0.000000,0.000000,1803.40,22.0,0.133082,0.893082
2,2,1Z23M00012463,82824ECB010,ﾜｲﾔﾘﾝｸﾞﾊｰﾈｽｺﾈｸﾀ,32,矢崎総業（株）,NaN,回収済,2023-12-08 00:00:00,1,...,10.492,0.82,3,0.0,0.000000,0.000000,374.75,11.0,0.256171,1.076171
3,3,1Z23M00011795,35847ECB010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,80,（株）メタルテック,NaN,回収済,2023-12-08 00:00:00,1,...,7.806,0.83,3,0.0,0.000000,0.000000,374.75,4.0,0.640427,1.470427
4,4,1Z23M00010125,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,NaN,回収済,2023-12-07 00:00:00,1,...,9.510,0.82,3,0.0,0.000000,0.000000,526.95,65.0,0.045545,0.865545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30614,30614,1Z23L00047027,35580ECB011,"ｱｷｭｴｰﾀｰASSY,ｼﾌﾄ ｺﾝﾄﾛｰﾙ",6,アイシン精機（株）,半田工場,回収済,2023-11-16 00:00:00,3,...,9.301,0.76,3,422.0,70.333333,8.791667,374.75,62.0,0.048032,0.808032
30615,30615,1Z23L00047021,35580ECB011,"ｱｷｭｴｰﾀｰASSY,ｼﾌﾄ ｺﾝﾄﾛｰﾙ",6,アイシン精機（株）,半田工場,回収済,2023-11-16 00:00:00,3,...,9.301,0.76,3,422.0,70.333333,8.791667,374.75,62.0,0.048032,0.808032
30616,30616,1Z23L00045676,35580ECB011,"ｱｷｭｴｰﾀｰASSY,ｼﾌﾄ ｺﾝﾄﾛｰﾙ",6,アイシン精機（株）,半田工場,回収済,2023-11-16 00:00:00,2,...,9.301,0.76,3,422.0,70.333333,8.791667,374.75,62.0,0.048032,0.808032
30617,30617,1Z23L00045674,35580ECB011,"ｱｷｭｴｰﾀｰASSY,ｼﾌﾄ ｺﾝﾄﾛｰﾙ",6,アイシン精機（株）,半田工場,回収済,2023-11-16 00:00:00,2,...,9.301,0.76,3,422.0,70.333333,8.791667,374.75,62.0,0.048032,0.808032


In [158]:
#週単位で加工

#中間成果物をダウンロード
zaikodata = pd.read_csv(file_path_zaikodata,encoding='shift_jis')
LTdata = pd.read_csv(file_path_LTdata_extract_with_tehaisu,encoding='shift_jis')

# '計測日時'をdatatime型に変換
zaikodata['計測日時'] = pd.to_datetime(zaikodata['計測日時'], errors='coerce')
# '回収日'をdatatime型に変換
LTdata['回収日時'] = pd.to_datetime(LTdata['回収日時'], errors='coerce')

#在庫推移データを週単位に加工
zaikodata.dropna(subset=['計測日時'], inplace=True)#土日は入庫出庫の履歴がないことがあるので欠損値は削除
zaikodata['年'] = zaikodata['計測日時'].dt.year
zaikodata['週番号'] = zaikodata['計測日時'].dt.isocalendar().week
#週ごとにグループ化し、'在庫数（箱）', '入庫数（箱）', '出庫数（箱）'を計算
weekly_zaikodata = zaikodata.groupby(['品番', '年', '週番号']).agg({
   '在庫数（箱）': 'median',
   '入庫数（箱）': 'sum',
   '出庫数（箱）': 'sum'
}).reset_index()

#LTデータを週単位に加工
LTdata['年'] = LTdata['回収日時'].dt.year
LTdata['週番号'] = LTdata['回収日時'].dt.isocalendar().week
#週ごとにグループ化し、各グループのLT中央値を計算➀
weekly_LTdata_1 = LTdata.groupby(['品番', '仕入先名','年','週番号','基準在庫日数', '基準在庫日数（基準在庫枚数込み）','基準在庫枚数','箱種類','納入回数（間隔）','納入回数（回数）','納入回数（遅れ）']).agg({'日量数（箱数）':'median','発注〜印刷LT': 'median','発注〜検収LT': 'median','発注〜順立装置入庫LT': 'median', '発注〜順立装置出庫LT': 'median', '発注〜回収LT': 'median'})
#週ごとにグループ化し、各グループの行数（かんばん数）をカウント➁
weekly_LTdata_2 = LTdata.groupby(['品番','仕入先名','年', '週番号','基準在庫日数', '基準在庫日数（基準在庫枚数込み）','基準在庫枚数','箱種類','納入回数（間隔）','納入回数（回数）','納入回数（遅れ）']).size().reset_index(name='回収かんばん数')
#➀と➁データを統合
merged_weekly_LTdata = pd.merge(weekly_LTdata_1, weekly_LTdata_2, on=['品番', '仕入先名','年','週番号'], how='outer')

#週単位の在庫推移データとLTデータを統合
weekly_data = pd.merge(weekly_zaikodata, merged_weekly_LTdata, on=['品番', '年', '週番号'], how='inner')

#検収と入庫の時間変なやつある

#各LTの計算
long = weekly_data.iloc[:,1]
count = 0
for i in range(len(long)):
    weekly_data.loc[i, '印刷検収LT'] = float(weekly_data.loc[i, '発注〜検収LT']) - float(weekly_data.loc[i, '発注〜印刷LT'])
    weekly_data.loc[i, '検収入庫LT'] = float(weekly_data.loc[i, '発注〜順立装置入庫LT'])-float(weekly_data.loc[i, '発注〜検収LT'])
    weekly_data.loc[i, '入庫出庫LT'] = float(weekly_data.loc[i, '発注〜順立装置出庫LT'])-float(weekly_data.loc[i, '発注〜順立装置入庫LT'])
    weekly_data.loc[i, '出庫回収LT'] = float(weekly_data.loc[i, '発注〜回収LT'])-float(weekly_data.loc[i, '発注〜順立装置出庫LT'])
    weekly_data.loc[i, '社内LT（検収〜回収LT）'] = float(weekly_data.loc[i, '発注〜回収LT'])-float(weekly_data.loc[i, '発注〜検収LT'])
    count = count + 1
print(count)

# Load the new CSV file into a DataFrame with 'Shift_JIS' encoding
pitch = pd.read_csv(file_path_pitch, encoding='Shift_JIS')

longi = len(weekly_data.loc[:,"品番"])
longj = len(pitch.loc[:,"仕入先名"])

for i in range(longi):
    for j in range(longj):
        if (weekly_data.loc[i, '仕入先名'] == pitch.loc[j,'仕入先名']):
            weekly_data.loc[i,"不等ピッチ係数"] = pitch.loc[j,"不等ピッチ係数【日】"]

# '基準在庫日数' から '社内LT（検収～回収LT）' を引いた新しい列 'LT_差分' を追加
weekly_data['社内LT/設計値LT'] = weekly_data['社内LT（検収〜回収LT）']/weekly_data['基準在庫日数']
weekly_data['社内LT/設計値LT（基準在庫枚数込み）'] = weekly_data['社内LT（検収〜回収LT）']/weekly_data['基準在庫日数（基準在庫枚数込み）']
weekly_data['順立装置在庫量/設計値MIN'] = weekly_data['在庫数（箱）']/(0.1*(weekly_data['日量数（箱数）']*weekly_data['納入回数（間隔）']*(1+weekly_data['納入回数（遅れ）'])/weekly_data['納入回数（回数）']))
weekly_data['順立装置在庫量/設計値MAX'] = weekly_data['在庫数（箱）']/(weekly_data['順立装置在庫量/設計値MIN'] + weekly_data['日量数（箱数）']/weekly_data['納入回数（回数）'] + weekly_data['日量数（箱数）']*weekly_data.loc[i,"不等ピッチ係数"])

display(weekly_data)

# 統合したデータを新しいCSVファイルに保存
with open(file_path_weekly_data, mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    weekly_data.to_csv(f)

172


,品番,年,週番号,在庫数（箱）,入庫数（箱）,出庫数（箱）,仕入先名,日量数（箱数）,発注〜印刷LT,発注〜検収LT,...,印刷検収LT,検収入庫LT,入庫出庫LT,出庫回収LT,社内LT（検収〜回収LT）,不等ピッチ係数,社内LT/設計値LT,社内LT/設計値LT（基準在庫枚数込み）,順立装置在庫量/設計値MIN,順立装置在庫量/設計値MAX
0,019128GA010,2023,49,26.0,0.0,0.0,石黒ゴム工業（株）,17.330000,0.320,1.940,...,1.620,2.500000e-01,2.050,0.720,3.020,0.29,2.822430,2.442669,17.411472,1.368417
1,01912ECB010,2023,49,7.0,0.0,0.0,石黒ゴム工業（株）,5.586667,0.110,1.940,...,1.830,3.000000e-01,2.990,0.260,3.550,0.29,3.317757,2.262505,14.541392,0.465008
2,01912ECB040,2023,49,30.0,0.0,0.0,愛産樹脂工業（株）,20.950000,0.000,4.390,...,4.390,8.881784e-16,2.180,0.405,2.585,0.00,3.858209,3.218850,16.618734,1.618197
3,01912ECB060,2023,49,1.0,0.0,0.0,新光ゴム工業（株）,0.870000,0.120,5.905,...,5.785,-5.400000e-01,1.915,2.750,4.125,0.00,5.288462,1.035877,4.954419,0.184063
4,1040043104R,2023,49,3.0,0.0,0.0,（株）ギフ加藤製作所,1.676000,0.185,2.045,...,1.860,4.700000e-01,4.570,1.610,6.650,0.12,7.388889,2.594085,7.276326,0.365937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,G1250ECB010,2023,49,2.0,0.0,0.0,（株）バルカー,2.800000,0.235,5.045,...,4.810,3.950000e-01,2.555,0.820,3.770,0.24,3.307018,1.863168,4.277160,0.343810
168,G1250ECC010,2023,49,4.0,0.0,0.0,（株）バルカー,3.100000,0.375,5.230,...,4.855,2.000000e-01,1.115,0.560,1.875,0.24,1.644737,0.855137,14.974730,0.262142
169,G1250ECE010,2023,49,10.0,0.0,0.0,（株）バルカー,8.916667,0.370,5.050,...,4.680,3.650000e-01,0.650,0.405,1.420,0.24,1.245614,0.958431,6.715541,0.860607
170,G9201ECB030,2023,49,26.0,0.0,0.0,佐藤工業（株）,45.000000,1.620,5.020,...,3.400,1.100000e-01,0.590,0.280,0.980,0.05,1.324324,1.218841,4.622222,1.434703


組立て稼働率
#shift-jisでダウンロード

In [180]:
# フォルダー内の全てのCSVファイルを見つける
csv_files_kumitateMB = [f for f in os.listdir(folder_path_kumitate) if f.endswith('.csv')]
#
print(f"{len(csv_files_kumitateMB)}つのファイルが見つかりました！")

# 統合結果を保存するための空のDataFrameを作成
merged_df_kumitate = pd.DataFrame()
# CSVファイルをDataFrameに読み込んでリストに保存
for file in csv_files_kumitateMB:
    file_path = os.path.join(folder_path_kumitate, file)
    df_kumitate = pd.read_csv(file_path, encoding='cp932')
    merged_df_kumitate = pd.concat([merged_df_kumitate, df_kumitate], ignore_index=True)

# 統合したデータを新しいCSVファイルに保存
with open(file_path_kumitate, mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_df_kumitate.to_csv(f)

#作成したDataFrameの内容を確認
display(merged_df_kumitate)

# データフレームのインデックスを日付型に変換
merged_df_kumitate['LINE_DATE'] = pd.to_datetime(merged_df_kumitate['LINE_DATE'])
#AVAILABLE_RATE.set_index('LINE_DATE', inplace=True)
# 'WEEK_NUMBER'列を追加し、'LINE_DATE'の週番号を計算
merged_df_kumitate['Week_Number'] = merged_df_kumitate['LINE_DATE'].dt.isocalendar().week
merged_df_kumitate['WEEKDAY'] = merged_df_kumitate['LINE_DATE'].dt.weekday

# 土日を除外
merged_df_kumitate2 = merged_df_kumitate[(merged_df_kumitate['WEEKDAY'] != 5) & (merged_df_kumitate['WEEKDAY'] != 6)]

#1Y
merged_df_kumitate3 = merged_df_kumitate2[(merged_df_kumitate2['KUMI_CD'] == "NM11") | (merged_df_kumitate2['KUMI_CD'] == "NM12")]

#作成したDataFrameの内容を確認
display(merged_df_kumitate3)

# 'AVAILABLE_RATE'列を週単位でリサンプリングし、平均を計算
weekly_average_available_rate = merged_df_kumitate3.groupby('Week_Number').agg({'OPERATION_RATE': 'median', 'KAKO_CNT': 'median'})

weekly_average_available_rate = weekly_average_available_rate.rename(columns={'OPERATION_RATE': '時間稼働率'})
weekly_average_available_rate = weekly_average_available_rate.rename(columns={'KAKO_CNT': '加工数'})

#AVAILABLE_RATE.head(300)
display(weekly_average_available_rate)

1つのファイルが見つかりました！


,KUMI_CD,LINE_CD,LINE_SUB_CD,KANRIBAN_CD,LINE_DATE,TYOKU_KBN,OFFICE_HOURS,WORKING_HOURS,LINE_START_TIME,LINE_END_TIME,...,KANRIBAN_STAFF_CD,CREATE_STAFF_CD,CREATE_TIME,UPDATE_STAFF_CD,UPDATE_TIME,FIRST_SERIAL_NO,KANSEI_SHOUNIN_STAFF,AUTHOR,PDF_NAME,HUB_CD
0,AP11,AS5561,A,M02,2023/04/11 00:00:00,2,690,0.0,< NULL >,< NULL >,...,SYS_TT,M02,2023/04/11 08:00:12,SYS_TT,2023/04/11 08:15:09,< NULL >,< NULL >,< NULL >,< NULL >,IA000
1,AP11,AS5561,A,M02,2023/04/12 00:00:00,2,690,0.0,< NULL >,< NULL >,...,SYS_TT,M02,2023/04/12 08:00:15,SYS_TT,2023/04/12 08:15:09,< NULL >,< NULL >,< NULL >,< NULL >,IA000
2,AP11,AS5561,A,M02,2023/04/13 00:00:00,2,690,0.0,< NULL >,< NULL >,...,SYS_TT,M02,2023/04/13 08:00:15,SYS_TT,2023/04/13 08:15:10,< NULL >,< NULL >,< NULL >,< NULL >,IA000
3,AP11,AS5561,A,M02,2023/04/14 00:00:00,2,690,0.0,< NULL >,< NULL >,...,SYS_TT,M02,2023/04/14 08:00:14,SYS_TT,2023/04/14 08:15:09,< NULL >,< NULL >,< NULL >,< NULL >,IA000
4,AP11,AS5561,A,M02,2023/04/15 00:00:00,2,690,0.0,2023/04/15 22:00:00,< NULL >,...,SYS_TT,M02,2023/04/15 08:00:09,SYS_TT,2023/04/15 08:15:09,< NULL >,< NULL >,< NULL >,< NULL >,IA000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379396,SQ11,AS3860,A,M01,2023/12/14 00:00:00,2,690,0.0,2023/12/14 22:00:00,< NULL >,...,SYS_TT,M01,2023/12/14 08:00:14,SYS_TT,2023/12/14 08:15:07,< NULL >,< NULL >,< NULL >,< NULL >,IS000
379397,SQ11,AS3860,A,M01,2023/12/15 00:00:00,1,750,486.0,2023/12/15 08:30:00,2023/12/15 18:30:00,...,1,M01,2023/12/15 08:00:16,1,2023/12/15 19:16:02,23M6Q05360,0119416,0141748,< NULL >,IS000
379398,SQ11,AS3860,A,M01,2023/12/15 00:00:00,2,690,0.0,2023/12/15 22:00:00,< NULL >,...,SYS_TT,M01,2023/12/15 08:00:16,SYS_TT,2023/12/15 08:15:07,< NULL >,< NULL >,< NULL >,< NULL >,IS000
379399,SQ11,AS3860,A,M01,2023/12/16 00:00:00,1,750,0.0,2023/12/16 08:30:00,< NULL >,...,SYS_TT,M01,2023/12/16 08:00:10,SYS_TT,2023/12/16 08:15:06,< NULL >,< NULL >,< NULL >,< NULL >,IS000


,KUMI_CD,LINE_CD,LINE_SUB_CD,KANRIBAN_CD,LINE_DATE,TYOKU_KBN,OFFICE_HOURS,WORKING_HOURS,LINE_START_TIME,LINE_END_TIME,...,CREATE_TIME,UPDATE_STAFF_CD,UPDATE_TIME,FIRST_SERIAL_NO,KANSEI_SHOUNIN_STAFF,AUTHOR,PDF_NAME,HUB_CD,Week_Number,WEEKDAY


,時間稼働率,加工数
Week_Number,,


In [76]:
#merged_weekly_LTdata.head(5)
#weekly_zaikodata.head(5)
#weekly_data.head(5)
#merged_df_LT2.loc[:,"回収月"].head(5)
merged_df_LT2.head(15)
#t = int(merged_df_LT2.loc[2,"回収月"])

,かんばんシリアル,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,納入便,発注日時,...,順立装置入庫日時,順立装置出庫日時,回収日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜回収LT,更新日時,回収月
0,1Z23M00016940,9036340A010,ﾍﾞｱﾘﾝｸﾞ ﾗｼﾞｱﾙﾎﾞｰﾙ,80,日本精工（株）,三河分室,回収済,2023-12-08,02,2023/12/07 06:00:11,...,2023/12/08 14:38:03,2023/12/09 03:55:59,2023-12-09 06:45:41,0.06,1.22,1.36,1.91,2.03,2023/12/09 06:45:41,12
1,1Z23M00015194,9036340A010,ﾍﾞｱﾘﾝｸﾞ ﾗｼﾞｱﾙﾎﾞｰﾙ,80,日本精工（株）,三河分室,回収済,2023-12-07,07,2023/12/06 18:06:56,...,2023/12/08 05:33:54,2023/12/08 23:52:00,2023-12-09 06:45:41,0.27,1.4,1.48,2.24,2.53,2023/12/09 06:45:41,12
2,1Z23M00012463,82824ECB010,ﾜｲﾔﾘﾝｸﾞﾊｰﾈｽｺﾈｸﾀ,32,矢崎総業（株）,NaN,回収済,2023-12-08,01,2023/12/06 10:42:00,...,2023/12/08 14:17:57,2023/12/09 03:36:05,2023-12-09 06:45:41,0.14,1.98,2.15,2.7,2.84,2023/12/09 06:45:41,12
3,1Z23M00011795,35847ECB010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,80,（株）メタルテック,NaN,回収済,2023-12-08,01,2023/12/06 06:45:37,...,2023/12/08 09:53:01,2023/12/09 00:37:03,2023-12-09 06:45:41,0.09,2.03,2.13,2.74,3,2023/12/09 06:45:41,12
4,1Z23M00010125,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,NaN,回収済,2023-12-07,01,2023/12/05 10:41:25,...,2023/12/07 14:24:18,2023/12/09 04:14:26,2023-12-09 06:45:41,0.14,1.98,2.15,3.73,3.84,2023/12/09 06:45:41,12
5,1Z23M00006992,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,NaN,回収済,2023-12-07,01,2023/12/05 10:41:25,...,2023/12/07 14:26:44,2023/12/09 04:14:17,2023-12-09 06:45:41,0.14,1.98,2.16,3.73,3.84,2023/12/09 06:45:41,12
6,1Z23M00006489,35441ECB010,ｽﾄﾚｰﾅｰｵｲﾙ,48,（株）ニフコ,名古屋事業所,回収済,2023-12-07,01,2023/12/05 09:47:52,...,2023/12/08 00:36:20,2023/12/09 05:26:51,2023-12-09 06:45:41,0.09,2.24,2.62,3.82,3.87,2023/12/09 06:45:41,12
7,1Z23M00005619,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,NaN,回収済,2023-12-07,01,2023/12/05 10:41:25,...,2023/12/07 14:27:15,2023/12/09 04:33:11,2023-12-09 06:45:41,0.14,1.98,2.16,3.74,3.84,2023/12/09 06:45:41,12
8,1Z23L00108973,82821ECB010,ｶﾊﾞ-ｺﾈｸﾀ-,400,石黒ゴム工業（株）,NaN,回収済,2023-12-07,01,2023/11/30 15:11:54,...,2023/12/07 16:23:32,2023/12/08 14:47:39,2023-12-09 06:45:41,0.01,6.76,7.05,7.98,8.65,2023/12/09 06:45:41,12
9,1Z23L00087001,9056439A006,ｼﾑ,100,（株）ムロコーポレーション,NaN,回収済,2023-11-29,01,2023/11/29 06:03:19,...,2023/11/30 06:13:07,2023/12/09 01:06:11,2023-12-09 06:45:41,0.1,0.86,1.01,9.79,10.03,2023/12/09 06:45:41,12


In [2]:
conda update matplotlib

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /anaconda/anaconda3

  added / updated specs:
    - matplotlib


The following packages will be REMOVED:

  importlib_resources-5.2.0-pyhd3eb1b0_1
  matplotlib-base-3.1.3-py37h9aa3819_0
  pkgutil-resolve-name-1.3.10-py37hecd8cb5_0

The following packages will be SUPERSEDED by a higher-priority channel:

  jsonschema         pkgs/main/osx-64::jsonschema-4.17.3-p~ --> pkgs/main/noarch::jsonschema-3.2.0-pyhd3eb1b0_2 None
  seaborn            pkgs/main/osx-64::seaborn-0.12.2-py37~ --> pkgs/main/noarch::seaborn-0.11.2-pyhd3eb1b0_0 None


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [3]:
conda update ipython

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /anaconda/anaconda3

  added / updated specs:
    - ipython


The following NEW packages will be INSTALLED:

  importlib_resourc~ pkgs/main/noarch::importlib_resources-5.2.0-pyhd3eb1b0_1 None
  matplotlib-base    pkgs/main/osx-64::matplotlib-base-3.1.3-py37h9aa3819_0 None
  pkgutil-resolve-n~ pkgs/main/osx-64::pkgutil-resolve-name-1.3.10-py37hecd8cb5_0 None

The following packages will be UPDATED:

  jsonschema         pkgs/main/noarch::jsonschema-3.2.0-py~ --> pkgs/main/osx-64::jsonschema-4.17.3-py37hecd8cb5_0 None
  seaborn            pkgs/main/noarch::seaborn-0.11.2-pyhd~ --> pkgs/main/osx-64::seaborn-0.12.2-py37hecd8cb5_0 None


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: 

In [75]:
merged_df.isnull().sum()
#merged_df.isnull().info()

かんばんシリアル           0
品番                 0
品名                 0
収容数                0
仕入先名               0
仕入先工場名         36386
ステータス              0
納入日                0
納入便                0
発注日時               0
印刷日時               0
検収日時               0
順立装置入庫日時           0
順立装置出庫日時           0
回収日時               0
発注〜印刷LT            0
発注〜検収LT            0
発注〜順立装置入庫LT        0
発注〜順立装置出庫LT        0
発注〜組立LT            0
発注〜回収LT            0
更新日時               0
dtype: int64

In [57]:
merged_df.dtypes

かんばんシリアル               object
伝票番号                   object
拠点所番地                  object
品番                     object
品名                     object
収容数                    object
仕入先名                   object
仕入先工場名                 object
ステータス                  object
納入日                    object
納入便                    object
発注日時                   object
印刷日時                   object
検収日時                   object
順立装置入庫日時               object
順立装置出庫日時               object
組立投入日時                 object
回収日時           datetime64[ns]
発注取消日時                 object
発注〜印刷LT                object
発注〜検収LT                object
発注〜順立装置入庫LT            object
発注〜順立装置出庫LT            object
発注〜組立LT                object
発注〜回収LT                object
更新日時                   object
長期滞留フラグ                object
dtype: object

In [70]:
merged_df.head(5)

,かんばんシリアル,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,納入便,発注日時,...,順立装置入庫日時,順立装置出庫日時,回収日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜組立LT,発注〜回収LT,更新日時
